In [ ]:
import requests
import entities.entities as en
from src.config import ORION_URL, NGSI_LD_ENDPOINT, headers,QL_NOTIFY
from src.load_data import df_DX_Session,df_cleaned_b103
import entities.fakeEntities as fe
import src.utils as ut
import json

ut.test_connection()

In [ ]:
# # Create a subscription

# def create_ql_subscription(entity_type):
#     subscription_data = {
#         "type": "Subscription",
#         "entities": [{"type": entity_type}],
#         "notification": {
#             "endpoint": {
#                 "uri": QL_NOTIFY
#             }
#         }
#     }
      
#     response = requests.post(
#         f"{ORION_URL}/ngsi-ld/v1/subscriptions/",
#         headers=headers,
#         json=subscription_data
#     )
    
#     print(f"{entity_type} - {response.status_code} - {response.text}")


# create_ql_subscription("ChargingSessionFake")
# create_ql_subscription("ChargingSession")
# create_ql_subscription("ChargingStation")
# create_ql_subscription("ChargingStationStatus")
# create_ql_subscription("ChargingPoint")
# create_ql_subscription("ChargingPointStatus")
# create_ql_subscription("E-Vehicle")

In [4]:
# #load real data

# B103_charging_points = []
# B103_charging_point_status = []

# for i in range(1, 11):
#     cp = en.B103_charging_point(i)
#     cps = en.B103_charging_point_status(i)

#     B103_charging_points.append(cp)
#     B103_charging_point_status.append(cps)

# Drahi_X_charging_session_entities= []
# Drahi_X_charging_session_entities = en.Drahi_X_charging_session(df_DX_Session)

# B103_charging_session_entities = []
# B103_charging_session_entities = en.B103_charging_session(df_cleaned_b103)

In [ ]:
# #insert real data

# entities = [
#     en.CS_001,
#     en.CS_002,
#     en.CS_001_S,
#     en.CS_002_S,
#     en.CP_001,
#     en.CP_001_S,
#     en.EV_001
# ]+ B103_charging_points + B103_charging_point_status 

# for entity in entities:
#     response = requests.post(
#         NGSI_LD_ENDPOINT,
#         json=entity,
#         headers=headers
#     )
#     print(response.status_code, response.text)

In [ ]:
# entities = Drahi_X_charging_session_entities
# for entity in entities:
#     response = requests.post(
#         NGSI_LD_ENDPOINT,
#         json=entity,
#         headers=headers
#     )
#     print(response.status_code, response.text)

In [ ]:
# entities = B103_charging_session_entities

# for entity in entities:
#     response = requests.post(
#         NGSI_LD_ENDPOINT,
#         json=entity,
#         headers=headers
#     )
#     print(response.status_code, response.text)

In [ ]:
#check subscription

response = requests.get(
    f"{ORION_URL}/ngsi-ld/v1/subscriptions/",
    headers=headers
)

print(response.status_code)
data = response.json()
print(json.dumps(data, indent=2, ensure_ascii=False))

In [44]:
# delete_subscription= requests.delete(f"{ORION_URL}/ngsi-ld/v1/subscriptions/urn:ngsi-ld:Subscription:ed11e79c-7399-11f0-b31e-9228036ea2f7")


# def delete_all_subscriptions():
#     try:
#         response = requests.get(f"{ORION_URL}/ngsi-ld/v1/subscriptions")
        
#         if response.status_code == 200:
#             subscriptions = response.json()
#             print(f"找到 {len(subscriptions)} 个订阅")
            
#             for subscription in subscriptions:
#                 subscription_id = subscription.get("id")
#                 if subscription_id:
#                     delete_response = requests.delete(
#                         f"{ORION_URL}/ngsi-ld/v1/subscriptions/{subscription_id}"
#                     )
                    
#                     if delete_response.status_code == 204:
#                         print(f"成功删除订阅: {subscription_id}")
#                     else:
#                         print(f"删除订阅失败: {subscription_id}, 状态码: {delete_response.status_code}")
#                         print(f"错误信息: {delete_response.text}")
#         else:
#             print(f"获取订阅列表失败，状态码: {response.status_code}")
#             print(f"错误信息: {response.text}")
            
#     except requests.exceptions.RequestException as e:
#         print(f"请求异常: {e}")

# delete_all_subscriptions()

In [8]:
# Generate fake sessions

batch_sessions = fe.generate_scenario_batch_sessions(
    total_count=300,
    batch_distribution={
        "random": 80,      
        "slot": 80,   
        "informed": 140   
    }
)

In [ ]:
#insert fake sessions
for scenario_name, sessions in batch_sessions.items():
    for entity in sessions:
        response = requests.post(
            NGSI_LD_ENDPOINT,
            json=entity,
            headers=headers
        )
        print(f"{scenario_name} - {response.status_code}: {response.text}")

In [48]:
# #there is an error

# QL_URL="http://localhost:8668/"

# response = requests.get(
#     f"{QL_URL}/v2/entities"
# )


# print(response.status_code)
# data = response.json()
# print(json.dumps(data, indent=2, ensure_ascii=False))

http://localhost:3000/login   admin/admin
http://localhost:4200/        crate

crate:542 / doc     
crate /  empty password

Variable type : query

chargingPoint: 
SELECT 'All' as __text, 'All' as __value
UNION ALL
SELECT DISTINCT refchargingpoint as __text, refchargingpoint as __value
FROM etchargingsessionfake 
WHERE refchargingpoint IS NOT NULL
ORDER BY __value

sessionStatus:
SELECT 'All' as __text, 'All' as __value
UNION ALL
SELECT DISTINCT sessionstatus as __text, sessionstatus as __value
FROM etchargingsessionfake 
WHERE sessionstatus IS NOT NULL
ORDER BY __value

SELECT COUNT(*) as "Total Sessions"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )

SELECT 
  CASE 
    WHEN EXTRACT(dow FROM time_index) IN (0, 6) THEN 'Weekend'
    ELSE 'Weekday'
  END as "Day Type",
  COUNT(*) as "Sessions",
  AVG(chargingduration) as "Average Duration",
  MIN(chargingduration) as "Min Duration",
  MAX(chargingduration) as "Max Duration"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
  AND chargingduration IS NOT NULL
GROUP BY 1

SELECT 
  refchargingpoint as "Charging point",
  SUM(energyconsumed) as "Total Energy (kWh)",
  AVG(energyconsumed) as "Avg Energy (kWh)",
  AVG(chargingduration/60.0) as "Avg Duration (hours)",
  COUNT(*) as "Sessions"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND sessionstatus = 'ended'
  AND energyconsumed > 0
GROUP BY refchargingpoint
ORDER BY "Total Energy (kWh)" DESC

SELECT 
  CASE 
    WHEN EXTRACT(hour FROM time_index) = 0 THEN '0-1h'
    WHEN EXTRACT(hour FROM time_index) = 1 THEN '1-2h'
    WHEN EXTRACT(hour FROM time_index) = 2 THEN '2-3h'
    WHEN EXTRACT(hour FROM time_index) = 3 THEN '3-4h'
    WHEN EXTRACT(hour FROM time_index) = 4 THEN '4-5h'
    WHEN EXTRACT(hour FROM time_index) = 5 THEN '5-6h'
    WHEN EXTRACT(hour FROM time_index) = 6 THEN '6-7h'
    WHEN EXTRACT(hour FROM time_index) = 7 THEN '7-8h'
    WHEN EXTRACT(hour FROM time_index) = 8 THEN '8-9h'
    WHEN EXTRACT(hour FROM time_index) = 9 THEN '9-10h'
    WHEN EXTRACT(hour FROM time_index) = 10 THEN '10-11h'
    WHEN EXTRACT(hour FROM time_index) = 11 THEN '11-12h'
    WHEN EXTRACT(hour FROM time_index) = 12 THEN '12-13h'
    WHEN EXTRACT(hour FROM time_index) = 13 THEN '13-14h'
    WHEN EXTRACT(hour FROM time_index) = 14 THEN '14-15h'
    WHEN EXTRACT(hour FROM time_index) = 15 THEN '15-16h'
    WHEN EXTRACT(hour FROM time_index) = 16 THEN '16-17h'
    WHEN EXTRACT(hour FROM time_index) = 17 THEN '17-18h'
    WHEN EXTRACT(hour FROM time_index) = 18 THEN '18-19h'
    WHEN EXTRACT(hour FROM time_index) = 19 THEN '19-20h'
    WHEN EXTRACT(hour FROM time_index) = 20 THEN '20-21h'
    WHEN EXTRACT(hour FROM time_index) = 21 THEN '21-22h'
    WHEN EXTRACT(hour FROM time_index) = 22 THEN '22-23h'
    WHEN EXTRACT(hour FROM time_index) = 23 THEN '23-24h'
  END as "Hour Range",
  CAST(SUM(energyconsumed) AS DOUBLE PRECISION) as "Total Energy (kWh)",
  EXTRACT(hour FROM time_index) as hour_num
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
  AND energyconsumed IS NOT NULL
GROUP BY EXTRACT(hour FROM time_index)
ORDER BY hour_num

SELECT 
  CASE 
    WHEN chargingduration < 30 THEN '< 30 min'
    WHEN chargingduration < 60 THEN '30-60 min'
    WHEN chargingduration < 120 THEN '1-2 h'
    WHEN chargingduration < 240 THEN '2-4 h'
    WHEN chargingduration < 480 THEN '4-8 h'
    ELSE '> 8 h'
  END as "duration",
  COUNT(*) as "Count"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
  AND chargingduration IS NOT NULL
GROUP BY 1
ORDER BY 
  CASE 
    WHEN (CASE 
            WHEN chargingduration < 30 THEN '< 30 min'
            WHEN chargingduration < 60 THEN '30-60 min'
            WHEN chargingduration < 120 THEN '1-2 h'
            WHEN chargingduration < 240 THEN '2-4 h'
            WHEN chargingduration < 480 THEN '4-8 h'
            ELSE '> 8 h'
          END) = '< 30 min' THEN 1
    WHEN (CASE 
            WHEN chargingduration < 30 THEN '< 30 min'
            WHEN chargingduration < 60 THEN '30-60 min'
            WHEN chargingduration < 120 THEN '1-2 h'
            WHEN chargingduration < 240 THEN '2-4 h'
            WHEN chargingduration < 480 THEN '4-8 h'
            ELSE '> 8 h'
          END) = '30-60 min' THEN 2
    WHEN (CASE 
            WHEN chargingduration < 30 THEN '< 30 min'
            WHEN chargingduration < 60 THEN '30-60 min'
            WHEN chargingduration < 120 THEN '1-2 h'
            WHEN chargingduration < 240 THEN '2-4 h'
            WHEN chargingduration < 480 THEN '4-8 h'
            ELSE '> 8 h'
          END) = '1-2 h' THEN 3
    WHEN (CASE 
            WHEN chargingduration < 30 THEN '< 30 min'
            WHEN chargingduration < 60 THEN '30-60 min'
            WHEN chargingduration < 120 THEN '1-2 h'
            WHEN chargingduration < 240 THEN '2-4 h'
            WHEN chargingduration < 480 THEN '4-8 h'
            ELSE '> 8 h'
          END) = '2-4 h' THEN 4
    WHEN (CASE 
            WHEN chargingduration < 30 THEN '< 30 min'
            WHEN chargingduration < 60 THEN '30-60 min'
            WHEN chargingduration < 120 THEN '1-2 h'
            WHEN chargingduration < 240 THEN '2-4 h'
            WHEN chargingduration < 480 THEN '4-8 h'
            ELSE '> 8 h'
          END) = '4-8 h' THEN 5
    ELSE 6
  END

SELECT 
  CASE 
    WHEN energyconsumed < 5 THEN '< 5 kWh'
    WHEN energyconsumed < 15 THEN '5-15 kWh'
    WHEN energyconsumed < 30 THEN '15-30 kWh'
    ELSE '> 30 kWh'
  END as "Energy Range",
  COUNT(*) as "Count"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
  AND energyconsumed IS NOT NULL
GROUP BY 1
ORDER BY 
  CASE 
    WHEN (CASE 
            WHEN energyconsumed < 5 THEN '< 5 kWh'
            WHEN energyconsumed < 15 THEN '5-15 kWh'
            WHEN energyconsumed < 30 THEN '15-30 kWh'
            ELSE '> 30 kWh'
          END) = '< 5 kWh' THEN 1
    WHEN (CASE 
            WHEN energyconsumed < 5 THEN '< 5 kWh'
            WHEN energyconsumed < 15 THEN '5-15 kWh'
            WHEN energyconsumed < 30 THEN '15-30 kWh'
            ELSE '> 30 kWh'
          END) = '5-15 kWh' THEN 2
    WHEN (CASE 
            WHEN energyconsumed < 5 THEN '< 5 kWh'
            WHEN energyconsumed < 15 THEN '5-15 kWh'
            WHEN energyconsumed < 30 THEN '15-30 kWh'
            ELSE '> 30 kWh'
          END) = '15-30 kWh' THEN 3
    ELSE 4
  END

SELECT 
  COUNT(*) as "Total session quantity",
  AVG(chargingduration) as "Average duration (min)",
  AVG(energyconsumed) as "Average energy consumed (Kwh)",
  AVG(energyconsumed / NULLIF(chargingduration, 0)) * 60 as "Average efficiency (Kwh/min)"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
  AND chargingduration > 0
  AND energyconsumed > 0

SELECT 
  $__timeGroup(time_index, $__interval) as time,
  SUM(energyconsumed) as "Total Energy (kWh)"
FROM etchargingsessionfake
WHERE $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
  AND energyconsumed IS NOT NULL
GROUP BY time
ORDER BY time

SELECT 
  date_trunc('day', time_index) as time,
  count(*) as "Session Quantity"
FROM 
  etchargingsessionfake 
WHERE 
  $__timeFilter(time_index)
  AND (
    CASE 
      WHEN $chargingPoint = 'All' THEN TRUE
      ELSE refchargingpoint = $chargingPoint
    END
  )
  AND (
    CASE 
      WHEN $sessionStatus = 'All' THEN TRUE
      ELSE sessionstatus = $sessionStatus
    END
  )
GROUP BY 
  date_trunc('day', time_index)
ORDER BY time